## import important packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import network_fcn as vae
import SiameseNetwork as smsn
import argparse
import pickle
from keras.layers import concatenate, Flatten
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from scipy import spatial
import os
from os.path import dirname, abspath
from utils import *
from itertools import combinations
import warnings
import Classify as cls
from modeltransfer import *
import matplotlib.colors as mcolors

## set GPU environment for tensorflow

In [ ]:
warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
physical_devices = tf.config.list_physical_devices('GPU') 
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

## initialize hyperparameters

In [ ]:
parser = argparse.ArgumentParser(description='')
parser.add_argument('--input_size', dest='input_size', default= (None, 1))
parser.add_argument('--batch_shape', dest='batch_shape', default=8)
parser.add_argument('--latent_shape', dest='latent_shape', default=128)
parser.add_argument('--para_shape', dest='para_shape', default=15)
parser.add_argument('--batch_size', dest='batch_size', type=int, default=64)
parser.add_argument('--epochs', dest='epochs', type=int, default=50)
parser.add_argument('--step_decay', dest='step_decay', type=int, default=5)
parser.add_argument('--patience', dest='patience', type=int, default=10)
parser.add_argument('--lr', dest='lr', type=float, default=0.00005)
parser.add_argument('--dec_loss', dest='dec_loss', default=vae.loss().loss_vae)
parser.add_argument('--dis_loss', dest='dis_loss', default=vae.loss().loss_bce)
parser.add_argument('--cls_loss', dest='cls_loss', default=vae.loss().loss_bce)
parser.add_argument('--beta_1', dest='beta_1', type=float, default=0.5)
parser.add_argument('--norm', dest='norm', default='batch_norm')
parser.add_argument('--n_down', dest='n_down', default=7)
parser.add_argument('--n_std', dest='n_std', default=0.15)
parser.add_argument('--batch', dest='batch', default=False)
parser.add_argument('--group', dest='group', default=False)
parser.add_argument('--n_group', dest='n_group', default=4)
args, unknown = parser.parse_known_args()

## import data and perform normalization, remove spectra with too low conrrelation compared to the other spectra

In [ ]:
metadata = pd.read_csv(dirname(abspath(os.getcwd())) + '/datasets/bacterial_SSP/metadata.csv')
spec = pd.read_csv(dirname(abspath(os.getcwd())) + '/datasets/bacterial_SSP/data.csv')
wn = np.array(spec.iloc[0,1:])
spec = np.array(spec.iloc[1:,1:])
labels = np.array(metadata['labels'])
batches = np.array(metadata['batches'])

ix_wn = np.asarray(range(len(wn)))[(wn>1850) & (wn<2750)]
ix_wn1 = np.asarray(range(len(wn)))[(wn<1800) | (wn>2800)]

for i in range(spec.shape[0]):
    spec[i,:] = spec[i, :]/np.max(spec[i, :])

ix = np.argwhere((labels != 'L-innocua') & (labels != 'P-stutzeri'))[:,0]
spec = spec[ix, :]
labels = labels[ix]
batches = batches[ix]

corr_all = np.mean(np.corrcoef(spec), 0)
i_good = corr_all>np.percentile(corr_all, 1)
spec = spec[i_good, :]
labels = labels[i_good]
batches = batches[i_good]

uni_labels = np.unique(labels)
dummy_y = np.zeros((len(labels), len(uni_labels)))
for i in range(len(uni_labels)):
    dummy_y[labels==uni_labels[i], i] = 1

n_spec_gen = 60

## perform spectra generation and model transfer
- generate spectra for different spectral variations and in case of different training data sizes
- perform model transfer, i.e, train with real data, and predict generated data.
- model transfer is achieved with different methods: EMSC, MS, and siamese network

In [ ]:
uni_batches = np.unique(batches)
index = np.array(range(len(uni_batches)))
for n_b in [2, 5, 8]:    ### number of batches for training

    accs = []; 
    b_tests = []; 
    methods = [];
    val_cors = []; 
    b_trains = [];
        
    bat = []
    for k in combinations(index, n_b):
        bat.append(k)
    bat = np.row_stack(bat)
    bat = bat[np.array(range(0, bat.shape[0], np.max([1, bat.shape[0]//9]))), :]
    
    for kk in range(bat.shape[0]):    ### index of batches to be used for training
        k = bat[kk,:]
        print(k)
        batch_sel = np.array([uni_batches[i] for i in k])    
        
        ix = [] 
        for b in batch_sel:
            if len(ix)<1:
                ix = (batches==b)
            else:
                ix = (ix) | (batches==b)
        ix = np.array(np.argwhere(ix==True)[:,0])
        
        ix_test_spec = np.array(list(set(range(spec.shape[0])) - set(ix)))

        ### perform classification
        model = cls.Classify(np.concatenate([spec[ix, :], spec[ix_test_spec, :]], axis=0), np.append(labels[ix], labels[ix_test_spec]), np.append(batches[ix], batches[ix_test_spec]))
        pred = model.model(range(len(ix)), range(len(ix), len(ix)+len(ix_test_spec)))
        acc = cls.cal_metric(labels[ix_test_spec], pred)
        accs = np.append(accs, acc) 
        b_tests = np.append(b_tests, batch_sel)
        methods = np.append(methods, 'org')
        b_trains = np.append(b_trains, kk)
        val_cors = np.append(val_cors, 2)
        
        ### prepare spectral pairs using spectra from same/different groups or batches
        x, y, g = prep_training(spec[ix,:], labels[ix], batches[ix], n_pairs=40000) 

        ### zero paddings to make the dimension of the spectra fit to the network
        downsampling = 2**args.n_down
        n_org = x.shape[1]
        n_res = (x.shape[1]//downsampling+1)*downsampling-x.shape[1]
        if n_res>0: 
            x_end = np.fliplr(x[:, -n_res:])
            y_end = np.fliplr(y[:, -n_res:])
            x = np.concatenate([x, x_end], axis=1)
            y = np.concatenate([y, y_end], axis=1)
        res1 = x.shape[1]//downsampling-args.para_shape
    
        ### split data into training and testing (validation) for the network training
        train_size = x.shape[0]//10*9
        ix_train = np.random.choice(range(x.shape[0]), train_size, replace=False)
        ix_test = list(set(range(x.shape[0])) - set(ix_train))

        ### parameters for the VAE network
        args.n_std = np.mean(np.std(snip(x[:, ix_wn], 10), axis=1))   ### estimate noise level from the data
        args.input_size = (x.shape[1], 1)       
        args.lr=0.00005
        args.epochs = 100
        model = vae.model_vae(args)    ### construct VAE network based on given parameters
        
        model.wn = wn.copy()
        model.x = x.copy()
        model.y = y.copy()
        model.g = g.copy()
        del x, y, g


        ### train VAE network 
        model.train_vae(ix_train, ix_test, res1, ratio=1e-4, f_model='_weights1.h5')
        if n_res>0: 
            x_end = np.fliplr(spec[:, -n_res:])
            spec1 = np.concatenate([spec, x_end], axis=1)

        ### test performance of the VAE by generating spectra of different similarity (controlled by 'cors')
        ix_gen = np.random.choice(ix, size=300, replace=False)
        y_gen, c_rea, c_gen, g_gen, b_gen = model.gen_cors(spec1[ix_gen, :], cors=np.asarray(np.linspace(90, 100, 20)/100.0), res1=res1)
    
        with open(dirname(abspath(os.getcwd())) + '/datasets/data_gen_all_' + str(kk) + '.pkl', 'wb') as f:
            pickle.dump([y_gen, c_gen, c_rea, g_gen, b_gen], f)

        ### plot the results of the generated spectra
        ix_sb_sg = (b_gen>0.5) & (g_gen>0.5)
        ix_db_sg = (b_gen<0.5) & (g_gen>0.5)
        ix_sb_dg = (b_gen>0.5) & (g_gen<0.5)
        ix_db_dg = (b_gen<0.5) & (g_gen<0.5)
    
        fig, ax = plt.subplots(2, 2, figsize=(10, 10))
        c_gen_sb_sg = c_gen[ix_sb_sg]
        c_gen_db_sg = c_gen[ix_db_sg]
        c_gen_db_dg = c_gen[ix_db_dg]
        c_gen_sb_dg = c_gen[ix_sb_dg]
    
        c_rea_sb_sg = c_rea[ix_sb_sg]
        c_rea_db_sg = c_rea[ix_db_sg]
        c_rea_db_dg = c_rea[ix_db_dg]
        c_rea_sb_dg = c_rea[ix_sb_dg]
    
        for c in np.asarray(np.linspace(90, 100, 20)/100.0):
            ix_plt = c_rea_sb_sg==c
            ax[0, 0].boxplot(c_gen_sb_sg[ix_plt]*100, positions=[c*100], widths=0.1, showfliers=False)
            ix_plt = c_rea_db_sg==c
            ax[0, 1].boxplot(c_gen_db_sg[ix_plt]*100, positions=[c*100], widths=0.1, showfliers=False)
            ix_plt = c_rea_db_dg==c
            ax[1, 0].boxplot(c_gen_db_dg[ix_plt]*100, positions=[c*100], widths=0.1, showfliers=False)
            ix_plt = c_rea_sb_dg==c
            ax[1, 1].boxplot(c_gen_sb_dg[ix_plt]*100, positions=[c*100], widths=0.1, showfliers=False)
        
        ax[0, 0].axline((np.min(c_rea_sb_sg)*100, np.min(c_rea_sb_sg)*100), slope=1, linewidth=1, color='gray', linestyle='--')
        ax[0, 0].set_xlabel('c-true')
        ax[0, 0].set_ylabel('c-generated')
        ax[0, 0].set_title('same group, same batch')
        ax[0, 0].set_xticks([90, 95, 100], [90, 95, 100]) 
        # ax[0, 1].scatter(c_rea_db_sg*100, c_gen_db_sg*100)
        ax[0, 1].axline((np.min(c_rea_db_sg)*100, np.min(c_rea_db_sg)*100), slope=1, linewidth=1, color='gray', linestyle='--')
        ax[0, 1].set_xlabel('c-true')
        ax[0, 1].set_ylabel('c-generated')
        ax[0, 1].set_title('same group, different batch')
        ax[0, 1].set_xticks([90, 95, 100], [90, 95, 100]) 
        ax[1, 0].axline((np.min(c_rea_db_dg)*100, np.min(c_rea_db_dg)*100), slope=1, linewidth=1, color='gray', linestyle='--')
        ax[1, 0].set_xlabel('c-true')
        ax[1, 0].set_ylabel('c-generated')
        ax[1, 0].set_title('different group, different batch')
        ax[1, 0].set_xticks([90, 95, 100], [90, 95, 100]) 
        ax[1, 1].axline((np.min(c_rea_sb_dg)*100, np.min(c_rea_sb_dg)*100), slope=1, linewidth=1, color='gray', linestyle='--')
        ax[1, 1].set_xlabel('c-true')
        ax[1, 1].set_ylabel('c-generated')
        ax[1, 1].set_title('different group, same batch')
        ax[1, 1].set_xticks([90, 95, 100], [90, 95, 100]) 
        plt.show()
        
        args.batch=False; args.lr=0.00001
        args.input_size = (spec.shape[1], 1)   
        args.epochs=200
        m_nn = smsn.SiameseNetwork(args)        #### initialize class substance from SiameseNetwork class
        m_nn.train_cls(spec[ix, :], dummy_y[ix, :], f_model='_weights1.h5')   #### train ordinary neural network


        #### preparing spectral pairs to train the siamese network
        n_pairs=40000
        x, y, cb, g = prep_training_cls(spec[ix,:], labels[ix], batches[ix], n_pairs, cb_shape=args.batch_shape)
        args.batch=False; args.group=False; args.lr=0.00001
        args.epochs=100
        m_smsn = smsn.SiameseNetwork(args)
        m_smsn.train_snet(x, y, g, cb, f_model='_weights1.h5')    ### siamese network with normal loss
            
        args.batch=True; args.group=False; args.lr=0.00001
        m_smsn_b = smsn.SiameseNetwork(args)
        print('\n siamese-b:\n')
        m_smsn_b.train_snet(x, y, g, cb, f_model='_weights1.h5')   ### siamese network with batch-based loss
    
        args.batch=False; args.group=True; args.lr=0.00001  
        m_smsn_g = smsn.SiameseNetwork(args)
        print('\n siamese-b:\n')
        m_smsn_g.train_snet(x, y, g, cb, f_model='_weights1.h5')    ### siamese network with group-based loss
    
        args.batch=True; args.group=True; args.lr=0.00001   ### siamese network with group- and batch-based loss
        m_smsn_gb = smsn.SiameseNetwork(args)
        print('\n siamese-b:\n')
        m_smsn_gb.train_snet(x, y, g, cb, f_model='_weights1.h5')
        
        smp_train = []
        for l in np.unique(labels[ix]):
            smp_train = np.append(smp_train, np.random.choice(ix[np.argwhere(labels[ix]==l)[:,0]], 10, replace=False))
        smp_train = np.array(smp_train, dtype='int32')
        
        n_spec_gen = 20
        for cc in np.asarray(np.linspace(90, 99, 9)):    #### generate spectra of different similarities as test data
            #### generate spectra of different similarities as test data
            y_gen, c_rea, c_gen, g_gen, b_gen = model.gen_mt(spec1[ix_gen,:], cors=np.asarray(np.linspace(cc, cc+1, n_spec_gen//4)/100.0), res1=res1)


            #### pick spectra belonging to the same batch as input spectra for testing
            spec_test = []
            label_test = [] 
            batch_test = [] 
            for i in range(len(ix_gen)):
                g_in_out = g_gen[(i*n_spec_gen):(i*n_spec_gen+n_spec_gen)]
                b_in_out = b_gen[(i*n_spec_gen):(i*n_spec_gen+n_spec_gen)]
                ix_pair = np.argwhere((g_in_out>0.5) & (b_in_out>0.5))[:,0]
                spec_test.append(y_gen[i*n_spec_gen+ix_pair, :1376])
                label_test = np.append(label_test, np.resize(labels[ix_gen[i]], len(ix_pair)))  
                batch_test = np.append(batch_test, np.resize('gen_Batch', len(ix_pair)))  
            
            spec_test = np.row_stack(spec_test)
    
            c_mod = cls.Classify(np.concatenate([spec[ix, :], spec_test], axis=0), np.append(labels[ix], label_test), np.append(batches[ix], batch_test))
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            b_tests = np.append(b_tests, batch_sel)
            methods = np.append(methods, 'sb')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### do model transfer based on MS
            c_mod = cls.Classify(np.concatenate([spec[ix, :], spec_test], axis=0), np.append(labels[ix], label_test), np.append(batches[ix], batch_test), True)
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            methods = np.append(methods, 'sb-MS')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
            
            ### do model transfer based on EMSC
            interference, _ = cls.get_meanspec(np.concatenate([spec[ix, :], spec_test], axis=0), labels=np.append(np.resize('org', len(ix)), batch_test))

            ### EMSC with one component
            cur_spec_corr1 = emsc(np.concatenate([spec[ix, :], spec_test], axis=0), degree=2, interferent=interference, interf_pca=1)['corrected']                
            c_mod = cls.Classify(cur_spec_corr1, np.append(labels[ix], label_test), np.append(batches[ix], batch_test))
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            methods = np.append(methods, 'sb-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### EMSC with two component
            cur_spec_corr2 = emsc(np.concatenate([spec[ix, :], spec_test], axis=0), degree=2, interferent=interference, interf_pca=2)['corrected']                
            c_mod = cls.Classify(cur_spec_corr2, np.append(labels[ix], label_test), np.append(batches[ix], batch_test))
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            b_tests = np.append(b_tests, batch_sel)
            methods = np.append(methods, 'sb-EMSC2')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with ordinary neural network
            pred_test, acc, min_acc, std_acc, method, b_test = m_nn.pred_cls(spec_test, label_test, batch_test, uni_labels)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-nn')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network (batch-based loss)
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_b.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-b')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network (group-based loss)
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_g.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-g')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network (group- and batch-based loss)
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_gb.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-gb')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network on spectra with EMSC 
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network (batch-loss) on spectra with EMSC 
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_b.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-b-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network (group-loss) on spectra with EMSC 
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_g.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-g-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)

            ### prediction with siamese network (batch- and group-loss) on spectra with EMSC 
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_gb.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'sb-snet-gb-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            #### pick spectra belonging to the different batch as input spectra for testing
            spec_test = [] 
            label_test = [] 
            batch_test = [] 
            for i in range(len(ix_gen)):
                g_in_out = g_gen[(i*n_spec_gen):(i*n_spec_gen+n_spec_gen)]
                b_in_out = b_gen[(i*n_spec_gen):(i*n_spec_gen+n_spec_gen)]
                ix_pair = np.argwhere((g_in_out>0.5) & (b_in_out<0.5))[:,0]
                spec_test.append(y_gen[i*n_spec_gen+ix_pair, :1376])
                label_test = np.append(label_test, np.resize(labels[ix_gen[i]], len(ix_pair)))  
                batch_test = np.append(batch_test, np.resize('gen_Batch', len(ix_pair)))  
            
            spec_test = np.row_stack(spec_test)
            
            c_mod = cls.Classify(np.concatenate([spec[ix, :], spec_test], axis=0), np.append(labels[ix], label_test), np.append(batches[ix], batch_test))
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            b_tests = np.append(b_tests, batch_sel)
            methods = np.append(methods, 'db')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### do model transfer based on MS
            c_mod = cls.Classify(np.concatenate([spec[ix, :], spec_test], axis=0), np.append(labels[ix], label_test), np.append(batches[ix], batch_test), True)
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            b_tests = np.append(b_tests, batch_sel)
            methods = np.append(methods, 'db-MS')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
            
            ### do model transfer based on EMSC
            interference, _ = cls.get_meanspec(np.concatenate([spec[ix, :], spec_test], axis=0), labels=np.append(np.resize('org', len(ix)), batch_test))
            ### EMSC with one component
            cur_spec_corr1 = emsc(np.concatenate([spec[ix, :], spec_test], axis=0), degree=2, interferent=interference, interf_pca=1)['corrected']                
            c_mod = cls.Classify(cur_spec_corr1, np.append(labels[ix], label_test), np.append(batches[ix], batch_test))
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            methods = np.append(methods, 'db-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
            
            ### EMSC with two component
            cur_spec_corr2 = emsc(np.concatenate([spec[ix, :], spec_test], axis=0), degree=2, interferent=interference, interf_pca=2)['corrected']                
            c_mod = cls.Classify(cur_spec_corr2, np.append(labels[ix], label_test), np.append(batches[ix], batch_test))
            pred = c_mod.model(range(len(ix)), range(len(ix), len(ix)+len(batch_test)))
            accs = np.append(accs, cls.cal_metric(label_test, pred)) 
            methods = np.append(methods, 'db-EMSC2')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
            
            ### prediction with ordinary neural network
            pred_test, acc, min_acc, std_acc, method, b_test = m_nn.pred_cls(spec_test, label_test, batch_test, uni_labels)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-nn')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network (batch-loss)
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_b.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-b')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network (group-loss)
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_g.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-g')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network (batch- and group-loss)
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_gb.pred_snet(spec[smp_train,:], labels[smp_train], spec_test, label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-gb')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network after EMSC correction
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network (batch-loss) after EMSC correction
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_b.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-b-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network (group-loss) after EMSC correction
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_g.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-g-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
            ### prediction with siamese network (batch- and group-loss) after EMSC correction
            pred_test, acc, min_acc, std_acc, method, b_test = m_smsn_gb.pred_snet(spec[smp_train,:], labels[smp_train], cur_spec_corr1[len(ix):,:], label_test, batch_test)
            accs = np.append(accs, np.mean(acc)) 
            methods = np.append(methods, 'db-snet-gb-EMSC1')
            b_trains = np.append(b_trains, kk)
            val_cors = np.append(val_cors, cc)
    
    with open('accs_mt1_gen_'+str(n_b)+'_batch.pkl', 'wb') as f:
        pickle.dump([accs, methods, b_trains, val_cors], f)    